In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
n_epochs = 10
batch_size_train = 64
batch_size_test = 1024
learning_rate = 0.01
momentum = 0.5
log_interval = 10
torch.manual_seed(1)


def generate_training_transforms():
  """
  Returns a transformation object which 1) converts images to tensors and 2) normalizes image pixel statistics.
  """
  return transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)) ])

def generate_testing_transforms():
  """
  Returns a transformation object which 1) converts images to tensors and 2) normalizes image pixel statistics.
  """
  return transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)) ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=generate_training_transforms())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_train,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=generate_testing_transforms())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size_test,
                                         shuffle=False, num_workers=2)

example = next(iter(trainloader))

type(example)

print("Example is of length {}".format(len(example)))
print("Type of first argument is {}".format(type(example[0])))
print("Type of second argument is {}".format(type(example[1])))

print("First argument has shape: {}".format(example[0].shape))
print("Second argument has shape {}".format(example[1].shape))
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3, 10)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x

network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

def train_epoch(network, trainloader, optimizer):
    network.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for data, target in trainloader:
        optimizer.zero_grad()
        output = network(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_samples += target.size(0)
        probabilities = F.softmax(output, dim=1)
        predicted_classes = torch.argmax(probabilities, dim=1)
        correct_predictions += (predicted_classes == target).sum().item()

    average_loss = total_loss / len(trainloader)
    accuracy = correct_predictions / total_samples
    return average_loss, accuracy


for epoch in range(n_epochs):
    average_loss, acc = train_epoch(network, trainloader, optimizer)
    print(f"Epoch {epoch + 1}/{n_epochs}: Average Loss = {average_loss:.4f}, Accuracy = {acc * 100:.2f}%")

Files already downloaded and verified
Files already downloaded and verified
Example is of length 2
Type of first argument is <class 'torch.Tensor'>
Type of second argument is <class 'torch.Tensor'>
First argument has shape: torch.Size([64, 3, 32, 32])
Second argument has shape torch.Size([64])
Epoch 1/10: Average Loss = 1.8640, Accuracy = 36.01%
Epoch 2/10: Average Loss = 1.7672, Accuracy = 39.73%
Epoch 3/10: Average Loss = 1.7387, Accuracy = 40.82%
Epoch 4/10: Average Loss = 1.7215, Accuracy = 41.29%
Epoch 5/10: Average Loss = 1.7099, Accuracy = 41.63%
Epoch 6/10: Average Loss = 1.7013, Accuracy = 41.96%
Epoch 7/10: Average Loss = 1.6948, Accuracy = 42.46%
Epoch 8/10: Average Loss = 1.6887, Accuracy = 42.53%
Epoch 9/10: Average Loss = 1.6845, Accuracy = 42.69%
Epoch 10/10: Average Loss = 1.6804, Accuracy = 42.89%
